<a href="https://colab.research.google.com/github/MohdIsmaeil/audioTranslation/blob/main/youtubeaudiobengali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install soundfile

In [ ]:
!pip install transformers youtube-transcript-api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.7 MB/s eta 0:00:00


In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import re

def get_video_id(url):
    # Extracts the video ID from a YouTube URL
    video_id = None
    match = re.match(r'(?:https?://)?(?:www\.)?(?:youtube\.com/watch\?v=|youtu\.be/)([a-zA-Z0-9_-]+)', url)
    if match:
        video_id = match.group(1)
    return video_id

def fetch_english_captions(video_url):
    video_id = get_video_id(video_url)
    if video_id is None:
        print("Invalid YouTube URL")
        return

    try:
        # Fetch the transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

        print("English Captions for Video:", video_url)

        # Print each caption's text in sequence
        for entry in transcript:
            print(entry['text'])

    except Exception as e:
        print(f"Error fetching captions: {e}")

if __name__ == "__main__":
    video_url = input("Enter YouTube video URL: ")
    fetch_english_captions(video_url)


Enter YouTube video URL: https://www.youtube.com/watch?v=WXl1oITICX0
English Captions for Video: https://www.youtube.com/watch?v=WXl1oITICX0
- Your debts don't disappear when you die.
Your soul will be held ransom
until your balance is settled.
Is it ever worth it for you to get ahead
by trampling the honor, the rights,
or the property of others?
And if you get ahead here using what Allah
hasn't actually permitted you to take,
what does it look like when Allah
settles that debt in the Barzakh?
The Prophet SallAllahu
'Alaihi Wasallam said,
"Verily, the worst act
of riba is to indulge
in attacking the reputation of
a Muslim without just cause".
If you let your tongue consume the honor
of people without care for them,
what about their property?
And it doesn't take much to see
that predatory financial
systems are killing our world.
They're perpetuating war,
poverty, recklessness,
and they're making the most disgusting
of human behaviors profitable.
Firstly, the order for the believer
is to

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import re
from transformers import pipeline
import torch
from transformers import VitsModel, VitsTokenizer
import soundfile as sf
import numpy as np

# Load the translation pipeline for English to Bengali
translator = pipeline("translation", model="facebook/m2m100_418M", tokenizer="facebook/m2m100_418M")

# Load the VITS TTS model and tokenizer for Bengali
model = VitsModel.from_pretrained("facebook/mms-tts-ben")
tokenizer = VitsTokenizer.from_pretrained("facebook/mms-tts-ben")

def get_video_id(url):
    # Extracts the video ID from a YouTube URL
    video_id = None
    match = re.match(r'(?:https?://)?(?:www\.)?(?:youtube\.com/watch\?v=|youtu\.be/)([a-zA-Z0-9_-]+)', url)
    if match:
        video_id = match.group(1)
    return video_id

def translate_text(text):
    # Translate English text to Bengali using Hugging Face pipeline
    # Specifying source and target language explicitly
    translated = translator(text, src_lang='en', tgt_lang='bn', max_length=512)
    return translated[0]['translation_text']

def generate_speech_for_caption(bengali_text):
    # Tokenize the Bengali text
    inputs = tokenizer(bengali_text, return_tensors="pt")
    input_ids = inputs["input_ids"]

    # Generate speech from the tokenized text
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract waveform from the model output
    speech = outputs["waveform"]
    return speech

def fetch_english_captions_and_translate(video_url):
    video_id = get_video_id(video_url)
    if video_id is None:
        print("Invalid YouTube URL")
        return

    try:
        # Fetch the transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

        print("English Captions for Video:", video_url)

        bengali_translations = []  # List to collect all Bengali translations
        captions_timestamps = []  # List to collect start and end times for each caption
        all_speech = []  # List to store individual speech waveforms

        for entry in transcript:
            english_text = entry['text']
            print(f"Original (English): {english_text}")

            # Translate the text to Bengali
            bengali_translation = translate_text(english_text)
            print(f"Translated (Bengali): {bengali_translation}")

            # Add the translation and timestamp to the list
            bengali_translations.append(bengali_translation)
            captions_timestamps.append((entry['start'], entry['start'] + entry['duration']))

            # Generate the speech for each caption
            speech_waveform = generate_speech_for_caption(bengali_translation)
            all_speech.append(speech_waveform)

        # Concatenate all speech waveforms to create the full audio
        final_speech = torch.cat(all_speech, dim=-1)

        # Save the final speech as a single .wav file
        audio_path = "output_bengali_speech.wav"
        sf.write(audio_path, final_speech.squeeze().cpu().numpy(), 22050)  # Assuming 22050 Hz sample rate
        print(f"Audio saved to {audio_path}")

        return audio_path

    except Exception as e:
        print(f"Error fetching captions: {e}")

if __name__ == "__main__":
    video_url = input("Enter YouTube video URL: ")
    fetch_english_captions_and_translate(video_url)


Device set to use cuda:0


Enter YouTube video URL: https://www.youtube.com/watch?v=WXl1oITICX0
English Captions for Video: https://www.youtube.com/watch?v=WXl1oITICX0
Original (English): - Your debts don't disappear when you die.
Translated (Bengali): আপনি মারা যখন আপনার দায়বদ্ধতা চলে যায় না।
Original (English): Your soul will be held ransom
until your balance is settled.
Translated (Bengali): তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি তুমি
Original (English): Is it ever worth it for you to get ahead
Translated (Bengali): আপনি কি কখনো এগিয়ে যাওয়ার মূল্যবান?
Original (English): by trampling the honor, the rights,
Translated (Bengali): শান্তি, অধিকার ও অধিকার প্রত্যাহার করে,
Original (English): or the property of others?
Translated (Bengali): অথবা অন্যদের মালিক?
Original (English): And if you get ahead here using what Allah
Translated (Bengali): আর যদি আপনি এখানে এগিয়ে যান, তাহলে আল্লাহ যা ব্যবহার করেন তা ব্যবহার করুন।
Original (English): hasn't actually permitted you to take,